In [1]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from datetime import datetime
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

In [2]:
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

def compute_spread(p1, p2, tf, x):
    data1 = get_rates(p1, tf, x)
    data2 = get_rates(p2, tf, x)
    merged = data1.join(data2, lsuffix="_x", rsuffix="_y")
    spread = merged['close_x'] - merged['close_y']
    return spread.dropna()

def adf_test(spread):
    '''Runs ADF test on a spread series'''
    result = adfuller(spread)
    return {'ADF Statistic': result[0], 'p-value': result[1], 'Critical Values': result[4]}

# Correlation Mean Reversion

If correlation diverges between two pairs of assets, price of the two assets will move oppositely. If correlation is mean reverting, you can profit from the correlation reverting back to its mean. Identify the leading and lagging pair for both high and low correlation. 
- Go long on the lagging
- Go short on the leader

In [95]:
def get_pair_correlations(symbol1, symbol2, window):
    s1 = str(symbol1)
    s2 = str(symbol2)
    symbol1 = get_rates(symbol1, mt5.TIMEFRAME_H4, 5000)
    symbol2 = get_rates(symbol2, mt5.TIMEFRAME_H4, 5000)

    combined_df = pd.concat([symbol1['close'].rename(f'{s1}_close'),
                             symbol2['close'].rename(f'{s2}_close')], axis=1)

    window_size = window  # Change this to the size of the window you want
    combined_df['rolling_corr'] = combined_df[f'{s1}_close'].rolling(window=window_size).corr(combined_df[f'{s2}_close'])
    # combined_df['rolling_corr'].iloc[0:200].plot()
    combined_df[f'{s1}_return'] = combined_df[f'{s1}_close'].pct_change()
    combined_df[f'{s2}_return'] = combined_df[f'{s2}_close'].pct_change()
    combined_df['diff'] = combined_df[f'{s1}_return'] - combined_df[f'{s2}_return']
    combined_df['rolling_corr_returns'] = combined_df['rolling_corr'].rolling(window=window_size).corr(combined_df['diff'])
    combined_df['shifted_rolling_corr_returns'] = combined_df['rolling_corr_returns'].shift(1)
    return combined_df.dropna()

In [115]:
df = get_pair_correlations('EURUSD.a', 'GBPUSD.a', 5)
df

,EURUSD.a_close,GBPUSD.a_close,rolling_corr,EURUSD.a_return,GBPUSD.a_return,diff,rolling_corr_returns,shifted_rolling_corr_returns
time,,,,,,,,
2020-08-13 12:00:00,1.18490,1.30980,0.998387,0.000895,0.000909,-0.000014,0.303580,0.477185
2020-08-13 16:00:00,1.18174,1.30796,0.957012,-0.002667,-0.001405,-0.001262,-0.009083,0.303580
2020-08-13 20:00:00,1.18146,1.30672,0.951366,-0.000237,-0.000948,0.000711,0.202063,-0.009083
2020-08-14 00:00:00,1.18095,1.30604,0.943589,-0.000432,-0.000520,0.000089,0.127828,0.202063
2020-08-14 04:00:00,1.18133,1.30586,0.924105,0.000322,-0.000138,0.000460,-0.251809,0.127828
...,...,...,...,...,...,...,...,...
2023-10-25 12:00:00,1.05749,1.21306,0.935835,-0.000964,-0.000643,-0.000321,-0.760610,-0.746943
2023-10-25 16:00:00,1.05746,1.21274,0.960348,-0.000028,-0.000264,0.000235,-0.763315,-0.760610
2023-10-25 20:00:00,1.05661,1.21113,0.992009,-0.000804,-0.001328,0.000524,-0.497463,-0.763315


In [86]:
grangercausalitytests(df[['shifted_rolling_corr_returns', 'EURUSD.a_return']], 10)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=22.9872 , p=0.0000  , df_denom=4987, df_num=1
ssr based chi2 test:   chi2=23.0010 , p=0.0000  , df=1
likelihood ratio test: chi2=22.9481 , p=0.0000  , df=1
parameter F test:         F=22.9872 , p=0.0000  , df_denom=4987, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=11.5986 , p=0.0000  , df_denom=4984, df_num=2
ssr based chi2 test:   chi2=23.2204 , p=0.0000  , df=2
likelihood ratio test: chi2=23.1665 , p=0.0000  , df=2
parameter F test:         F=11.5986 , p=0.0000  , df_denom=4984, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=7.6572  , p=0.0000  , df_denom=4981, df_num=3
ssr based chi2 test:   chi2=23.0039 , p=0.0000  , df=3
likelihood ratio test: chi2=22.9510 , p=0.0000  , df=3
parameter F test:         F=7.6572  , p=0.0000  , df_denom=4981, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=5.7557  , p=0.

{1: ({'ssr_ftest': (22.987163742824315, 1.6783018197278783e-06, 4987.0, 1),
   'ssr_chi2test': (23.00099199452443, 1.6191782707352778e-06, 1),
   'lrtest': (22.948143747600625, 1.6643084186514316e-06, 1),
   'params_ftest': (22.987163742823498, 1.6783018197286533e-06, 4987.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (11.598555813403655, 9.429687495721975e-06, 4984.0, 2),
   'ssr_chi2test': (23.22038320749231, 9.073145024268906e-06, 2),
   'lrtest': (23.166512793974107, 9.3208531198904e-06, 2),
   'params_ftest': (11.598555813403648, 9.429687495721975e-06, 4984.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (7.657210147971926, 4.1941005182295216e-05, 4981.0, 3),
   'ssr_chi2test': (23.003913401777137, 4.0307242345585886e-05, 3),
   'lrtest': (22.95103061919508, 4.1343320132206484e-05, 3),
   'params_ftest': (7.657210147971774, 4.1941005182321664e-05, 4981.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0

### Based on the above findings:
Next steps:
1. Picking a direction; if 'diff' is positive, EURUSD may outperform GBPUSD, vice versa. 
2. You can forecast each symbol individually since there's a granger-casual relationship present.

Models to try:
- ARIMA
- LSTM 

In [97]:
def send_order(symbol, side, lot, comment):

    if side.lower() == 'sell':
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    elif side.lower() == 'buy':
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "deviation": 5,
        "magic": 234000,
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    result

In [137]:
def close_position(position):

    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": 'Regres Close',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result = mt5.order_send(request)
    
def check_close_position(trade_position):
    from datetime import datetime
    from datetime import datetime, timedelta
    # Your logic to close the position
    ticket = trade_position.ticket
    time = datetime.fromtimestamp(trade_position.time)
    print(time)
    symbol = trade_position.symbol
    comment = trade_position.comment
    # print(f"Checking {symbol} of {comment} to close")
    # Sample trade position (replace this with the actual object you get)
    position = {
        'ticket': ticket,
        'time': time,  # This would be the actual Unix timestamp
        'Symbol': symbol,
        'Comment': comment,
    }

    local_to_utc_offset = timedelta(hours= +3)

    # Get the current time in GMT+3
    current_time_gmt_plus_3 = datetime.now() 

    # Convert it to UTC
    current_time_utc = current_time_gmt_plus_3 + local_to_utc_offset

    # Convert to Unix timestamp
    current_unix_time = int(current_time_utc.timestamp())

    # Calculate the time difference in seconds
    time_difference = current_unix_time - int(i.time)
    
    # Check if 4 hours or more have passed (4 hours = 4 * 60 * 60 seconds)
    if time_difference >= 14400:
        print(f"Time difference is {round((time_difference / 60),2) / 60} hrs. Closing {symbol} of {comment}")
        close_position(trade_position)
    else:
        print(f"Position {symbol} of {comment} has been open for less than 4 hours. Time open: {round(round((time_difference / 60),2) / 60), 4} hrs")

In [119]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

model_diff = ARIMA(df['diff'], order=(1,0,0), exog=df['shifted_rolling_corr_returns'])
model_diff_fit = model_diff.fit()
yhat_diff = model_diff_fit.forecast(steps=1, exog=np.array([[df['shifted_rolling_corr_returns'].iloc[-2]]]))

if yhat_diff.values > 0:
    send_order('EURUSD.a', 'sell', 1.50, 'Correl_Rev')
    send_order('GBPUSD.a', 'buy', 1.30, 'Correl_Rev')
elif yhat_diff.values < 0:
    send_order('EURUSD.a', 'buy', 1.50, 'Correl_Rev')
    send_order('GBPUSD.a', 'sell', 1.30, 'Correl_Rev')

In [120]:
model_diff = ARIMA(df['EURUSD.a_return'], order=(1,0,0), exog=df['shifted_rolling_corr_returns'])
model_diff_fit = model_diff.fit()
yhat_diff = model_diff_fit.forecast(steps=1, exog=np.array([[df['shifted_rolling_corr_returns'].iloc[-2]]]))

if yhat_diff.values > 0:
    send_order('EURUSD.a', 'buy', 1.50, 'Correl_Rev')
elif yhat_diff.values < 0:
    send_order('EURUSD.a', 'sell', 1.50, 'Correl_Rev')

In [121]:
model_diff = ARIMA(df['GBPUSD.a_return'], order=(1,0,0), exog=df['shifted_rolling_corr_returns'])
model_diff_fit = model_diff.fit()
yhat_diff = model_diff_fit.forecast(steps=1, exog=np.array([[df['shifted_rolling_corr_returns'].iloc[-2]]]))

if yhat_diff.values > 0:
    send_order('GBPUSD.a', 'buy', 1.30, 'Correl_Rev')
elif yhat_diff.values < 0:
    send_order('GBPUSD.a', 'sell', 1.30, 'Correl_Rev')

In [141]:
positions = mt5.positions_get()
for i in positions:
    if 'H1' in i.comment:
        print(f"Checking {i.symbol}")
        check_close_position(i)

Checking EURUSD.a
2023-10-26 15:10:14
Position EURUSD.a of H1Correl_Rev_D has been open for less than 4 hours. Time open: 0.013666666666666666 hrs
Checking GBPUSD.a
2023-10-26 15:10:15
Position GBPUSD.a of H1Correl_Rev_D has been open for less than 4 hours. Time open: 0.013333333333333334 hrs
Checking EURUSD.a
2023-10-26 15:10:16
Position EURUSD.a of H1Correl_Rev_EU has been open for less than 4 hours. Time open: 0.013000000000000001 hrs
Checking GBPUSD.a
2023-10-26 15:10:18
Position GBPUSD.a of H1Correl_Rev_GU has been open for less than 4 hours. Time open: 0.0125 hrs
